<p align="center">
  <img src="https://huggingface.co/speakleash/Bielik-7B-Instruct-v0.1/raw/main/speakleash_cyfronet.png">
</p>

### Przygotowanie środowiska

In [ ]:
#%%capture

!pip install accelerate -q
!pip install -i https://pypi.org/simple/ bitsandbytes -q


In [ ]:
import gc
import torch

In [ ]:
import warnings
warnings.filterwarnings("ignore") # tylko na potrzeby naszego demo ;)

### Inicjalizacja modelu (4 bit)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TextStreamer

device = "cuda"
model_name = 'speakleash/Bielik-7B-Instruct-v0.1'


temperature = 1.0
max_tokens = 500
top_k = 200
top_p = 1

load_4_bit = True


tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

quantization_config = None

if load_4_bit:
  quantization_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_compute_dtype=torch.bfloat16
  )

model = AutoModelForCausalLM.from_pretrained(model_name,
                                            torch_dtype=torch.bfloat16,
                                            quantization_config=quantization_config
                                            )



Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:02<00:00,  1.04it/s]


### Funkcja pomocnicza do generowania odpowiedzi

In [ ]:
import uuid
import json

temperature = 0.7
max_tokens = 256
top_k = 100
top_p = 1

def generate(prompt, system = None):

  messages = []

  if system:
    messages.append( {"role": "system", "content": system })

  messages.append( {"role": "user", "content": prompt })


  input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt")
  model_inputs = input_ids

  if torch.cuda.is_available():
    model_inputs = input_ids.to(device)

  # generowanie odpowiedzi
  outputs = model.generate(model_inputs,
                               streamer = streamer,
                               max_new_tokens=max_tokens,
                               do_sample=True if temperature else False,
                               temperature = temperature,
                               top_k = top_k,
                               top_p = top_p)

  # zapis do pliku
  filename = f"{str(uuid.uuid4())[:8]}.txt"

  with open(filename, "w") as file:
      content = {
          "prompt": messages,
           "output": tokenizer.batch_decode(outputs, skip_special_tokens=False)
      }
      json.dump(content, file, ensure_ascii=False, indent=4)




#### Test

In [ ]:
generate("Hej, jesteś tam?")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[/INST] [INST] Hej, jesteś tam? [/INST]  Cześć! Tak, jestem tutaj. W czym mogę Ci pomóc?


In [ ]:
generate("Hej, czym się zajmujesz?", system = "Jesteś programistą baz danych, który chce rozwijać swoje kompetencje w obszarze AI")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Cześć! Jako programista baz danych rozwijam się w obszarze sztucznej inteligencji. Na co dzień zajmuję się implementacją AI w różnych branżach, od medycyny po finanse. Opracowuję modele statystyczne, testuję nowe algorytmy, a także doradzam klientom w kwestiach związanych z wykorzystaniem AI w ich projektach. Moim celem jest promowanie AI jako efektywnego narzędzia do rozwiązywania złożonych problemów. Cieszę się, że mogę pomagać firmom i instytucjom w wykorzystywaniu SI do osiągania przewag konkurencyjnych.


# Przykład 1

Po prostu **"Hello World"**, a bardziej **"Witaj świecie!"**

In [ ]:
temperature = 1.0
#max_tokens = 256

prompt = "Witaj świecie!"
generate(prompt)



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Witaj świecie! Cieszymy się, że do nas dołączyłeś.


# Przykład 2

Zadawanie pytań otwartych taki LLM-owy jeden z dziesięciu.

In [ ]:
temperature = 0.1
max_tokens = 512

prompt = "Co to jest fotosynteza?"

In [ ]:
generate(prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Fotosynteza to proces, w którym rośliny, algi i niektóre bakterie przekształcają energię słoneczną w chemiczną energię w postaci glukozy, wykorzystując do tego chlorofil - zielony barwnik obecny w chloroplastach. W trakcie tego procesu rośliny pobierają dwutlenek węgla z powietrza i wodę z gleby, a następnie emitują tlen jako produkt uboczny. Fotosynteza jest kluczowa dla życia na Ziemi, ponieważ dostarcza niezbędnej energii do utrzymania złożonych ekosystemów i łańcucha pokarmowego.


# Przykład 3

Praca na tekście

In [ ]:
temperature = 0.1
max_tokens = 512

In [ ]:
review = "Przy waszych odsetkach to tylko chleb i margaryna!"
prompt = f"Czy podana opinia jest pozytywna czy negatywne i wyjaśnij to obszernie.\n\n###OPINIA:\n{review}"

generate(prompt)

In [ ]:
system = "Pracujesz jako specjalista od zarządzania jakością w banku i badasz doświadczenia klientów. Analizujesz opinie, wskazujesz pozytywne aspekty, na które zwrócili uwagę klienci oraz obszary do poprawy. Odpowiadasz w języku polskim."

generate(review, system)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Dziękujemy za wyrażenie swojej opinii. Jesteśmy świadomi, że może być wiele do poprawy w obszarze doświadczeń naszych klientów. Cieszymy się, że zwróciłeś uwagę na te obszary i doceniamy Twoje zaangażowanie w doskonalenie naszych usług. Będziemy nadal monitorować Twoje sugestie i dążyć do zapewnienia jeszcze lepszych doświadczeń finansowych. Dziękujemy za korzystanie z naszych usług i życzymy miłego dnia!


In [ ]:
temperature = 0.2

review = """Aplikacja działała świetnie i jest już w języku angielskim...​
Jednak od tygodnia zawiesza się, nie można się zalogować, używam tylko blika i akceptuję płatności...​
Kiedy to zostanie rozwiązane?"""

prompt = f"Czy podana opinia jest pozytywna czy negatywne? Uzasadnij odpowiedź, wymień w punktach aspekty pozytywne oraz wymagające poprawy. \n\n###OPINIA:\n{review}"

generate(prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Opinia jest mieszana, ponieważ podaje zarówno aspekty pozytywne, jak i negatywne. 

Aspekty pozytywne: 
1. Aplikacja działa świetnie 
2. Jest już w języku angielskim 

Aspekty wymagające poprawy: 
1. Od tygodnia się zawiesza 
2. Nie można się zalogować 
3. Używany jest tylko Blik do płatności 
4. Brak informacji o problemie ze strony aplikacji 
5. Pytanie o termin rozwiązania problemu, które nie otrzymało odpowiedzi


# Przykład 4
Praca z dokumentami

In [ ]:
import requests
from bs4 import BeautifulSoup
import re

def get_page_content(url = 'https://hiperautomatyzacja.pl'):

  response = requests.get(url)
  html_content = response.text
  soup = BeautifulSoup(html_content, 'html.parser')
  plain_text = soup.get_text()
  #clean_text = re.sub(r'\s+', ' ', plain_text).strip()
  clean_text = plain_text
  return clean_text

In [ ]:
temperature = 0.1
max_tokens = 512

system = "Dokonaj streszczenia podanego tekstu strony internetowej. Pisz w języku polskim."
prompt = get_page_content("https://hiperautomatyzacja.pl")

generate(prompt, system = system)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 HiperAutomatyzacja.pl to strona internetowa poświęcona konferencji na temat automatyzacji i AI w skali Enterprise. Organizator konferencji oferuje nieodpłatny udział w 8 edycji konferencji, podczas której prezentowane będą case studies z Polski. Konferencja odbędzie się w dniach 10-12 czerwca 2024 r. w Warszawie.

Konferencja skierowana jest do menedżerów i specjalistów ds. automatyzacji i robotyzacji procesów biznesowych. Uczestnicy konferencji będą mieli możliwość zobaczenia 18 case studies z Polski.

Organizator konferencji zapewnia dostęp do nagrań z konferencji dla uczestników odpłatnych. Ponadto, uczestnicy otrzymają dostęp do szkolenia on-line "Moc Inteligentnych Procesów" na rok od daty zakupu.

Konferencja jest częścią większego projektu - Liderzy.AI, mającego na celu budowanie polskiej społeczności hiperautomatyzacji i AI.


In [ ]:
prompt = get_page_content("https://hiperautomatyzacja.pl/program?s=7")
generate(prompt, system = system)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 HiperAutomatyzacja.pl to program skupiający się na zastosowaniu automatyzacji i sztucznej inteligencji w biznesie. Wydarzenie ma na celu pomóc firmom w transformacji cyfrowej i zoptymalizowaniu procesów. W ramach programu organizowane są konferencje, podczas których eksperci prezentują najnowsze trendy i praktyki w branży. Uczestnicy mają również możliwość wymiany doświadczeń i nawiązania cennych kontaktów biznesowych. Partnerem strategicznym programu jest firma Morele.net, a partnerem merytorycznym - PZU. Partnerami konferencji są m.in. ING Hubs Poland, XTB, Raben Group, i Ergo Ex.


In [ ]:
system = "Wymień warsztaty dotyczące AI oraz prowadzących z imienia, nazwiska z poniższego tekstu strony internetowej."
prompt = system + "\n\nTEKST:" + get_page_content("https://hiperautomatyzacja.pl/program?s=7")
generate(prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Oto warsztaty dotyczące AI wymienione w tekście:

1. Zbigniew Kępiński - HiperAutomatyzacja.pl - Hiperautomatyzacja w logistyce
2. Tomasz Gawron - XTB - Automatyzujące w skali Enterprise
3. Maciej Zyzak - ING Hubs Poland - Product Manager (Big Data & AI)
4. Adam Rataj - Morele.net - Change Director (PZU IT Delivery Manager (Big Data & AI))
5. Maciej Rudziński - PZU - IT Delivery Manager (Big Data & AI)
6. Tomasz Łopuchowicz - ISS World Services - Automation & RPA Manager
7. Artur Szylkin - ISS World Services - Automation, AI governance & Integration Domain Architect
8. Krzysztof Karaszewski - Międzynarodowy Bank - Product Manager (Big Data & AI)
9. Paweł Bulowski - Ergo ex Ergo, In transition - Wersja 2.0 dokumentu Dobrych Praktyk Robotyzacji Procesów wydanego przez Związek Banków Polskich
10. Radosław Repczyński - Credit Agricole Bank Polska - Senior Business Process Manager (Bielik w praktyce, czyli jak zbudować czatbota korzystając z otwartego polskiego modelu językowego)


In [ ]:
system = """Wymień warsztaty dotyczące AI z poniższego tekstu strony internetowej. Zwróć w postaci JSON według szablonu
[
  {
    'trener':...,
    'temat':...,
    'godzina': ...
    }
    ,
] """

prompt = system + "\n\nTEKST:" + get_page_content("https://hiperautomatyzacja.pl/program?s=7")
generate(prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 {
 "trener": {
   "name": "Zbigniew Kępiński",
   "event": "HiperAutomatyzacja.pl",
   "date": "09.05.2023",
   "time": "09:10",
   "location": "Online"
 },
 "trener": {
   "name": "Tomasz Gawron",
   "event": "HiperAutomatyzacja.pl",
   "date": "09.05.2023",
   "time": "10:00",
   "location": "Online"
 },
 "trener": {
   "name": "Maciej Zyzak",
   "event": "HiperAutomatyzacja.pl",
   "date": "09.05.2023",
   "time": "10:50",
   "location": "Online"
 },
 "trener": {
   "name": "Maciej Rudziński",
   "event": "HiperAutomatyzacja.pl",
   "date": "09.05.2023",
   "time": "11:30",
   "location": "Online"
 },
 "trener": {
   "name": "Adam Rataj",
   "event": "HiperAutomatyzacja.pl",
   "date": "09.05.2023",
   "time": "12:50",
   "location": "Online"
 },
 "trener": {
   "name": "Tomasz Łopuchowicz",
   "event": "HiperAutomatyzacja.pl",
   "date": "09.05.2023",
   "time": "13:30",
   "location": "Online"
 },
 "trener": {
   "name": "Artur Szylkin",
   "event": "HiperAutomatyzacja.pl",
   "d

In [ ]:
max_tokens = 1024
temperature = 0.1

system = "Pracujesz w agencji rekrutacyjnej. Pomagasz dużej firmie z branży finansowej zatrudnić menedżera do spraw AI. Dokonaj oceny podanego kandydata i wypisz jego mocne strony."
prompt = get_page_content("https://hiperautomatyzacja.pl/o-organizatorze")
generate(prompt, system = system)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Mocne strony kandydata:

1. Głęboka wiedza i doświadczenie w zarządzaniu projektami: Kandydat posiada ponad 15 lat doświadczenia w zarządzaniu projektami w różnych branżach, co pozwala mu na efektywne planowanie, koordynację i kontrolę projektów.

2. Silne umiejętności analityczne i strategiczne: Kandydat posiada wysoko rozwinięte umiejętności analizy biznesowej i strategicznego myślenia, co pozwala mu na identyfikowanie możliwości i ryzyk oraz tworzenie efektywnych planów działania.

3. Wszechstronność technologiczna: Kandydat jest biegły w różnych technologiach, w tym w zarządzaniu projektami, analizie danych, zarządzaniu zespołami zdalnymi oraz tworzeniu i wdrażaniu strategii cyfrowych.

4. Umiejętność budowania relacji i zespołów: Kandydat posiada wysoko rozwinięte umiejętności interpersonalne i przywódcze, co pozwala mu na skuteczne budowanie zespołów i efektywną komunikację z partnerami biznesowymi.

5. Skoncentrowanie na rezultatach: Kandydat jest silnie skoncentrowany na osiąg